In [ ]:
import feedparser
import psycopg2
from psycopg2 import sql


In [ ]:

# Configurações do RSS e banco de dados
RSS_URL = "https://news.google.com/rss/topics/CAAqBwgKMP-Jlwsw6sevAw?hl=pt-BR&gl=BR&ceid=BR:pt"

DB_CONFIG = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': '12345678',
    'host': 'localhost',
    'port': 5432
}

def fetch_tech_news():
    """Busca notícias de tecnologia do Google News via RSS"""
    feed = feedparser.parse(RSS_URL)
    if 'entries' in feed:
        return feed.entries
    else:
        print("Nenhuma notícia encontrada ou erro ao acessar o feed.")
        return []

def save_news_to_db(news_list):
    """Salva a lista de notícias no banco PostgreSQL"""
    try:
        connection = psycopg2.connect(**DB_CONFIG)
        cursor = connection.cursor()

        for news in news_list:
            titulo = news.get('title')
            conteudo = news.get('summary', '')  # Resumo da notícia
            imagem = news.get('media_content', [{}])[0].get('url') if 'media_content' in news else None

            # Inserir notícia na tabela
            query = sql.SQL("""
                INSERT INTO noticias (titulo, conteudo, imagem)
                VALUES (%s, %s, %s)
                ON CONFLICT (titulo) DO NOTHING;  -- Evita duplicatas
            """)
            cursor.execute(query, (titulo, conteudo, imagem))

        connection.commit()
        print("Notícias salvas com sucesso!")
    except Exception as e:
        print(f"Erro ao salvar no banco de dados: {e}")
    finally:
        if connection:
            cursor.close()
            connection.close()

def main():
    """Fluxo principal do script"""
    print("Buscando notícias de tecnologia do Google News...")
    news_list = fetch_tech_news()
    if news_list:
        print(f"{len(news_list)} notícias encontradas. Salvando no banco...")
        save_news_to_db(news_list)
    else:
        print("Nenhuma notícia encontrada.")

if __name__ == "__main__":
    main()


Buscando notícias de tecnologia...
Nenhuma notícia encontrada.
